<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین سوم (موتور جستجوی اخبار)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        دسترسی به داده‌ها و مدل‌های ذخیره شده
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        به دلیل حجم بالای فایل‌های ذخیره شده، از قرار دادن آن‌ها در پوشه نوتبوک صرف نظر کرده و با اجرای قطعه کد زیر، از طریق گوگل درایو به آن‌ها دسترسی پیدا می‌کنیم.
    </p>
</div>

In [1]:
# import sys
# from google.colab import drive
# drive.mount('/content/drive')
# address = "University/Term 6/MIR/Homeworks/HW3"
# sys.path.append(f"/content/drive/My Drive/{address}")
# %cd /content/drive/My\ Drive/$address

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/University/Term 6/MIR/Homeworks/HW3


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [ ]:
!pip install bs4
!pip install tqdm
!pip install pandas
!pip install requests
!pip install hazm
!pip install unidecode
!pip install fasttext
!pip install scikit-learn
!pip install pandas
!pip install nltk
!pip install torch
!pip install transformers
!pip install sentence_transformers
!pip install faiss-cpu
!pip install faiss-gpu

In [1]:
import os
import re
import csv
import hazm
import nltk
import json
import torch
import pickle
import zipfile
import requests
import fasttext
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from bs4 import BeautifulSoup
from string import punctuation
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٦٨ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [2]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [3]:
class Scraper:

    def __init__(self, current_year, current_month):
        self.current_year = current_year
        self.current_month = current_month

    def get_URL_content(self, URL):
        while True:
            try:
                return requests.get(URL, timeout=5).content
            except:
                pass

    def generate_page_URL(self, page_index, category, year, month):
        tp = {'Politics': 6, 'World': 11, 'Economy': 10, 'Society': 5, 'City': 7,
              'Sport': 9, 'Science': 20, 'Culture': 26, 'IT': 718, 'LifeSkills': 21}[category]
        return f'https://www.hamshahrionline.ir/archive?pi={page_index}&tp={tp}&ty=1&ms=0&mn={month}&yr={year}'

    def get_page_URLs_by_time(self, category, year, month):
        URLs = []
        page_index = 1
        while True:
            URL = self.generate_page_URL(page_index, category, year, month)
            content = self.get_URL_content(URL)
            if re.findall('pagination', str(content)):
                URLs.append(URL)
                page_index += 1
            else:
                break
        return URLs

    def get_page_URLs_since(self, category, year, month):
        URLs = []
        with tqdm() as pbar:
            while True:
                if month > 12:
                    month = 1
                    year += 1
                pbar.set_description(f'[{category}] [Extracting page URLs] [Date: {year}/{month}]')
                URLs_by_time = self.get_page_URLs_by_time(category, year, month)
                if URLs_by_time:
                    for URL in URLs_by_time:
                        URLs.append(URL)
                    month += 1
                elif self.current_year > year or (self.current_year == year and self.current_month > month):
                    month += 1
                else:
                    break
        return URLs

    def get_news_URLs_since(self, category, year, month):
        news_URLs = []
        page_URLs = self.get_page_URLs_since(category, year, month)
        with tqdm(page_URLs) as pbar:
            for page_URL in pbar:
                content = self.get_URL_content(page_URL)
                soup = BeautifulSoup(content, 'html5lib')
                for item in soup.findAll('li', attrs={'class': 'news'}):
                    URL = item.find('div', attrs={'class': 'desc'}).find('h3').find('a')['href']
                    URL = 'https://www.hamshahrionline.ir' + URL
                    news_URLs.append(URL)
                pbar.set_description(f'[{category}] [Extracting news URLs] [{len(news_URLs)} news until now]')
        return news_URLs

    def parse_news(self, URL, category):
        try:
            content = self.get_URL_content(URL)
            soup = BeautifulSoup(content, 'html.parser')
            date = soup.find('div', {'class': 'col-6 col-sm-4 col-xl-4 item-date'}).span.text.strip()
            title = soup.find('div', {'class': 'item-title'}).h1.text.strip()
            intro = soup.find('p', {'class': 'introtext', 'itemprop': 'description'}).text.strip()
            body = soup.find('div', {'class': 'item-text', 'itemprop': 'articleBody'}).text.strip()
            return {
                'date': date,
                'title': title,
                'intro': intro,
                'body': body,
                'category': category,
            }
        except:
            return None

    def scrape(self, from_year, from_month):
        categories = ['Politics', 'World', 'Economy', 'Society', 'City',
                      'Sport', 'Science', 'Culture', 'IT', 'LifeSkills']
        news = []
        for category in categories:
            URLs = self.get_news_URLs_since(category, from_year, from_month)
            with tqdm(URLs) as pbar:
                pbar.set_description(f'[{category}] [Scraping news]')
                for URL in pbar:
                    news.append(self.parse_news(URL, category))
        news = list(filter(None, news))
        pd.DataFrame(news).to_csv(f'dataset.csv', encoding='utf-8')
        with zipfile.ZipFile('dataset.zip', 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.write('dataset.csv')
        os.remove('dataset.csv')

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Scraper ایجاد شده و شروع به دریافت و ذخیره‌سازی داده‌ها می‌کند. خبرهای دریافت شده همگی مربوط به قرن جدید، از سال ١٤٠٠ به بعد هستند.
    </p>
</div>

In [4]:
scraper = Scraper(current_year=1401, current_month=3)
#scraper.scrape(from_year=1400, from_month=1)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        بعد از ذخیره شدن داده‌ها در فایل فشرده dataset.zip، آن‌ها را از این فایل استخراج کرده و وارد برنامه می‌کنیم. با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [5]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    os.remove('dataset.csv')
    return dataset


dataset = pd.DataFrame(read_dataset_from_file())

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [6]:
def display_dataset_info():
    global CATEGORIES, dataset

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for _, data in dataset.iterrows():
        length_dict[data['category']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,68362
1,سیاسی,15798
2,جهانی,2895
3,اقتصادی,8900
4,اجتماعی,13585
5,شهری,3853
6,ورزشی,8348
7,علمی,3190
8,فرهنگی,6512
9,فناوری اطلاعات,437


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. این مراحل عبارتند از:
    </p>
</div>

<div dir="auto" align="justify">
    <li style='direction:rtl;text-align:justify;'>
        نرمال سازی داده‌ها (normalize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف لینک‌ها (remove_links)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف نشانه‌های نگارشی (remove_punctuations)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        واحد سازی داده‌ها (word_tokenize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف کلمات نامعتبر (remove_invalid_words)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف ایست‌واژه‌ها (remove_stopwords)
    </li>
</div>

In [7]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_invalid_words(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]

In [8]:
def save_preprocessed_texts(texts, path="Preprocessed_texts.pickle"):
    with open(path, "wb") as file:
        pickle.dump(texts, file)


def load_preprocessed_texts(path="Preprocessed_texts.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)


def data_to_text(data):
    return ' '.join([data['title'], data['intro'], data['body']]).lower()


def get_preprocessed_texts(dataset, preprocessor, mode, save=False):
    preprocessed_texts = []
    if mode == 'process':
        texts = [data_to_text(data) for _, data in dataset.iterrows()]
        preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    if mode == 'load':
        preprocessed_texts = load_preprocessed_texts()
    if save:
        save_preprocessed_texts(preprocessed_texts)
    return preprocessed_texts

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Preprocessor ایجاد کرده و شروع به پیش پردازش داده‌ها می‌کنیم، یا داده‌های پیش‌پردازش‌شده‌ی ذخیره‌شده را از فایل مربوطه بازیابی می‌کنیم.
    </p>
</div>

In [9]:
preprocessor = Preprocessor(stopwords_path='stopwords.txt')
preprocessed_texts = get_preprocessed_texts(dataset, preprocessor, mode='load', save=False)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با توجه به این که حجم مجموعه داده‌ها باعث ایجاد محدودیت حافظه و زمان برای بعضی از مدل‌ها می‌شود، ناچار هستیم کسری از مجموعه‌ی داده‌ها را به عنوان ورودی برای یادگیری به آن‌ها بدهیم. با اجرای قطعه کد زیر، یک مجموعه داده‌ی کوچک‌تر از روی مجموعه داده‌ی اصلی ایجاد می‌کنیم.
    </p>
</div>

In [12]:
def get_mini_dataset(len_each_category=400):
    global CATEGORIES, dataset

    mini_dataset = []
    for category in CATEGORIES.keys():
        dataset_by_category = dataset.loc[dataset['category'] == category]
        length = min(len_each_category, dataset_by_category.shape[0])
        mini_dataset.append(dataset_by_category.sample(length, random_state=1))

    mini_dataset = pd.concat(mini_dataset).reset_index(drop=True)
    texts = [data_to_text(data) for _, data in mini_dataset.iterrows()]
    mini_preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    return mini_dataset, mini_preprocessed_texts


mini_dataset, mini_preprocessed_texts = get_mini_dataset()

100%|██████████| 4000/4000 [00:25<00:00, 153.97it/s]


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٣. دریافت اخبار مرتبط با کوئری کاربر
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این مرحله با استفاده از چهار مدل متفاوت زبانی، شروع به یادگیری روی خبرهای استخراج شده می‌کنیم و در انتها، نتیجه اجرای ده کوئری را بر روی هر یک مشاهده می‌کنیم. کوئری‌های مورد استفاده در این بخش در زیر آمده‌اند.
    </p>
</div>

In [10]:
QUERIES = [
    "نتیجه توافق برجام",
    "حمله ارتش روسیه به اوکراین",
    "افزایش نرخ تورم در کشور",
    "آمار فوتی‌های کرونا",
    "اقدامات شهرداری تهران",
    "صعود ایران به جام جهانی",
    "بیماری آبله میمونی",
    "جشنواره فیلم فجر",
    "رشد رمزارزها",
    "روش پخت غذا",
]

<div>
    <h4 style='direction:rtl;text-align:justify;'>
        ٤. مدل مبتنی بر میانگین وزن‌دار بردارهای تعبیه، مانند FastText
    </h4>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این بخش، از کتاب‌خانه‌ی fasttext استفاده می‌کنیم و با استفاده از روش skipgram و یک فایل شامل داده‌های موجود اخبار، مدل را به صورت unsupervised به یادگیری وامی‌داریم. پارامترهای استفاده شده‌ی دیگر هم در کد مشخصند؛ به طور مثال، کمینه و بیشینه n را برابر ۲ و ۵ در نظر گرفته‌ایم. پس از انجام یادگیری، میانگین امبدینگ‌ها را در یک آرایه‌ی نامپای ذخیره می‌کنیم. تمام این کارها با صدا زدن متد prepare با حالت train
        رخ می‌دهند. همچنین می‌توانیم این متد را با حالت load صدا بزنیم تا یک مدل ذخیره شده را از فایل بازیابی کنیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        علاوه بر این، می‌توانیم با استفاده از متدهای save_FastText_model و load_FastText_model این مدل و جزئیاتش را ذخیره یا بازیابی کنیم. البته اگر بخواهیم عمل ذخیره را بلافاصله پس از پایان یادگیری انجام دهیم هم می‌توانیم مقدار True را به آرگومان save در متد prepare بدهیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        پس از پایان یادگیری یا بازیابی مدل یادگرفته شده از فایل ذخیره شده، می‌توانیم با استفاده از متد predict به تعداد دلخواه خبر مرتبط به درخواستمان را به دست بیاوریم. برای این کار، مدلمان بردارهای مربوط به درخواست را به دست می‌آورد و با محاسبه‌ی کسینوس‌ها و به دست آوردن میزان شباهت بین درخواست و خبرهای موجود، شبیه‌ترین خبرها را خروجی می‌دهد.
    </p>
</div>

In [36]:
class FastText:

    def __init__(self, preprocessor=None, method='skipgram'):
        self.method = method
        self.mean_embed = []
        self.model = None
        self.preprocessor = preprocessor

    def train(self, texts):
        with open('FastText_train.txt', 'w', encoding='utf-8') as file:
            file.write('\n'.join(list(map(lambda doc: ' '.join(doc), texts))))
        self.model = fasttext.train_unsupervised('FastText_train.txt', self.method, minn=2, maxn=5, wordNgrams=10)
        os.remove('FastText_train.txt')
        self.mean_embed = list(map(lambda doc:
                                   np.mean(list(map(lambda word:
                                                    self.model.get_word_vector(word), doc)), axis=0), texts))
        self.mean_embed = np.array(self.mean_embed)

    def predict(self, query, dataset, k):
        if self.preprocessor:
            query = self.preprocessor.preprocess(query)
        if type(query) == str:
            query = query.split()
        query_embed = np.mean(list(map(lambda word: self.model.get_word_vector(word), query)), axis=0)
        dataset_sim = np.array(list(map(lambda doc: self.cosine_sim(query_embed, doc), self.mean_embed)))
        idx = np.argsort(-dataset_sim)
        return dataset.iloc[list(idx[:k])]

    def expand_query(self, query, k=5, lambda_0=1, lambda_1=1):
        if self.preprocessor:
            query = self.preprocessor.preprocess(query)
        if type(query) == str:
            query = query.split()
        query_embed = np.mean(list(map(lambda word: self.model.get_word_vector(word), query)), axis=0)
        dataset_sim = np.array(list(map(lambda doc: self.cosine_sim(query_embed, doc), self.mean_embed)))
        idx = np.argsort(-dataset_sim)
        relevant_docs_mean = np.mean(self.mean_embed[list(idx[:k]), :], axis=0)
        irrelevant_docs_mean = np.mean(self.mean_embed[list(idx[-k:]), :], axis=0)
        final_embed = query_embed + lambda_0 * relevant_docs_mean - lambda_1 * irrelevant_docs_mean
        return final_embed
    
    def predict_query_vector(self, query_embed, dataset, k):
        dataset_sim = np.array(list(map(lambda doc: self.cosine_sim(query_embed, doc), self.mean_embed)))
        idx = np.argsort(-dataset_sim)
        return dataset.iloc[list(idx[:k])]

    def cosine_sim(self, query, doc):
        return np.dot(query, doc) / (np.linalg.norm(query) * np.linalg.norm(doc))

    def save_FastText_model(self, path='FastText_model.bin'):
        self.model.save_model(path)
        np.save('FastText_mean_embed.npy', self.mean_embed)

    def load_FastText_model(self, path="FastText_model.bin"):
        self.model = fasttext.load_model(path)
        self.mean_embed = np.load('FastText_mean_embed.npy')

    def prepare(self, dataset, mode, save=False):
        if mode == 'train':
            self.train(dataset)
        if mode == 'load':
            self.load_FastText_model()
        if save:
            self.save_FastText_model()

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از مدل FastText ایجاد کرده و شروع به یادگیری داده‌ها و یا خواندن مدل ذخیره شده از روی فایل می‌کنیم.
    </p>
</div>

In [28]:
FastText_model = FastText()
FastText_model.prepare(preprocessed_texts, mode='load', save=False)

In [34]:
qu = FastText_model.expand_query('گوشی')

0.96251285


In [35]:
FastText_model.predict_query_vector(qu, dataset, k=5)

,date,title,intro,body,category
63420,شنبه ۳۱ اردیبهشت ۱۴۰۱ - ۱۰:۴۲,معرفی بهترین گوشی تا قیمت ۸ میلیون تومان,شما می توانید با بررسی معروف ترین و بهترین گوش...,از سوی دیگر در سال های اخیر قیمت مربوط به گوشی...,IT
23850,چهارشنبه ۲۴ آذر ۱۴۰۰ - ۰۸:۵۴,جدول | جدیدترین قیمت گوشی اپل در بازار | حداقل...,در این گزارش قیمت انواع گوشی اپل موجود در بازا...,به گزارش همشهری آنلاین به نقل از باشگاه خبرنگا...,Economy
23876,سه‌شنبه ۲۳ آذر ۱۴۰۰ - ۰۹:۵۶,جدول | جدیدترین قیمت انواع گوشی‌ | ارزان‌ترین ...,در این گزارش قیمت انواع گوشی شیائومی در بازار ...,به گزارش همشهری آنلاین به نقل از باشگاه خبرنگا...,Economy
26393,جمعه ۲۶ فروردین ۱۴۰۱ - ۰۸:۱۹,قیمت‌های عجیب انواع گوشی آیفون | موبایل ۶۰ میل...,در بازار موبایل انواع گوشی‌های آیفون ۱۳ و ۱۲ ر...,به گزارش همشهری آنلاین به نقل از خبرآنلاین، در...,Economy
23971,پنجشنبه ۱۸ آذر ۱۴۰۰ - ۰۹:۲۷,جدول | جدیدترین قیمت انواع گوشی‌ | معرفی گران‌...,پرچمدار گران‌ترین گوشی‌ در بازار موبایل گوشی س...,به گزارش همشهری آنلاین به نقل از خبرآنلاین، در...,Economy


<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        از این که وقت ارزشمند خود را برای بررسی این نوتبوک صرف کرده‌اید، صمیمانه سپاسگزاریم. (:
    </p>
</div>